# **\[FRAUD\]** 데이터정리 시도(df50 X범주_auto)

김보람  
2023-10-11

# imports

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import networkx as nx
import sklearn
import xgboost as xgb

# sklearn
from sklearn import model_selection # split함수이용
from sklearn import ensemble # RF,GBM
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

# gnn
import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import GCNConv


# autogluon
from autogluon.tabular import TabularDataset, TabularPredictor

In [15]:
def down_sample_textbook(df):
    df_majority = df[df.is_fraud==0].copy()
    df_minority = df[df.is_fraud==1].copy()
    df_maj_dowsampled = sklearn.utils.resample(df_majority, n_samples=len(df_minority), replace=False, random_state=42)
    df_downsampled = pd.concat([df_minority, df_maj_dowsampled])
    return df_downsampled

def compute_time_difference(group):
    n = len(group)
    result = []
    for i in range(n):
        for j in range(n):
            time_difference = abs(group.iloc[i].trans_date_trans_time.value - group.iloc[j].trans_date_trans_time.value)
            result.append([group.iloc[i].name, group.iloc[j].name, time_difference])
    return result


class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(1, 16)
        self.conv2 = GCNConv(16,2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [16]:
fraudTrain = pd.read_csv("~/Desktop/fraudTrain.csv").iloc[:,1:]

In [17]:
fraudTrain = fraudTrain.assign(trans_date_trans_time= list(map(lambda x: pd.to_datetime(x), fraudTrain.trans_date_trans_time)))
fraudTrain

## 데이터정리

In [40]:
_df1 = fraudTrain[fraudTrain["is_fraud"] == 0].sample(frac=0.20, random_state=42)
_df2 = fraudTrain[fraudTrain["is_fraud"] == 1]
df02 = pd.concat([_df1,_df2])
df02.shape

In [41]:
df50 = down_sample_textbook(df02)
df50.shape

In [42]:
df50 = df50.reset_index()

In [43]:
N = len(df50)

------------------------------------------------------------------------

# autogluon1: amt

In [34]:
df50 = df50[["amt","is_fraud"]]
df50_tr,df50_test = sklearn.model_selection.train_test_split(df50, random_state=42)

## A. 데이터

In [35]:
tr = TabularDataset(df50_tr)
tst = TabularDataset(df50_test)

## B. predictor 생성

In [36]:
predictr = TabularPredictor("is_fraud")

No path specified. Models will be saved in: "AutogluonModels/ag-20231011_124800/"

## C.적합(fit)

In [37]:
predictr.fit(tr) 

Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231011_124800/"
AutoGluon Version:  0.8.2
Python Version:     3.8.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #26~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Jul 13 16:27:29 UTC 2
Disk Space Avail:   746.82 GB / 982.82 GB (76.0%)
Train Data Rows:    9009
Train Data Columns: 1
Label Column: is_fraud
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
    2 unique label values:  [1, 0]
    If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
    Available Memory:                    14427.76 MB
    Train Data (

In [59]:
predictr.leaderboard()

                  model  score_val  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2   0.902331       0.051067  7.458454                0.001309           0.513563            2       True         14
1         LightGBMLarge   0.894562       0.001894  0.414943                0.001894           0.414943            1       True         13
2              CatBoost   0.894562       0.001981  0.653966                0.001981           0.653966            1       True          7
3               XGBoost   0.893452       0.003316  0.114061                0.003316           0.114061            1       True         11
4              LightGBM   0.892342       0.003488  0.343734                0.003488           0.343734            1       True          4
5        NeuralNetTorch   0.885683       0.005610  5.186066                0.005610           5.186066            1       True         12
6        KNeighborsUnif   0.877913

------------------------------------------------------------------------

# autogluon2: amt, distace

In [46]:
df50 = df50[["amt","distance_km", "is_fraud"]]

In [47]:
df50_tr,df50_test = sklearn.model_selection.train_test_split(df50, random_state=42)

## A. 데이터

In [48]:
tr = TabularDataset(df50_tr)
tst = TabularDataset(df50_test)

## B. predictor 생성

In [49]:
predictr = TabularPredictor("is_fraud")

No path specified. Models will be saved in: "AutogluonModels/ag-20231011_125208/"

## C.적합(fit)

In [50]:
predictr.fit(tr) 

Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231011_125208/"
AutoGluon Version:  0.8.2
Python Version:     3.8.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #26~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Jul 13 16:27:29 UTC 2
Disk Space Avail:   746.64 GB / 982.82 GB (76.0%)
Train Data Rows:    9009
Train Data Columns: 2
Label Column: is_fraud
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
    2 unique label values:  [1, 0]
    If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
    Available Memory:                    14301.5 MB
    Train Data (O

In [51]:
predictr.leaderboard()

                  model  score_val  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0              CatBoost   0.899001       0.001843  0.508956                0.001843           0.508956            1       True          7
1   WeightedEnsemble_L2   0.899001       0.003138  1.012600                0.001295           0.503644            2       True         14
2              LightGBM   0.891232       0.002272  0.223588                0.002272           0.223588            1       True          4
3               XGBoost   0.889012       0.003450  0.180574                0.003450           0.180574            1       True         11
4            LightGBMXT   0.887902       0.006675  0.331872                0.006675           0.331872            1       True          3
5        NeuralNetTorch   0.885683       0.005594  4.485880                0.005594           4.485880            1       True         12
6         LightGBMLarge   0.877913

------------------------------------------------------------------------

# autogluon3: amt, time, distace

In [45]:
df50['trans_date_trans_time'] = pd.to_datetime(df50['trans_date_trans_time'])
df50['trans_date_trans_time'] = (df50['trans_date_trans_time'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

category_map = {category: index for index, category in enumerate(df50['category'].unique())}
df50['category'] = df50['category'].map(category_map)

def haversine(lat1, lon1, lat2, lon2):
    # 지구의 반지름 (미터)
    radius = 6371.0

    # 라디안으로 변환
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    # Haversine 공식 계산
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = radius * c

    return distance

customer_lat = df50['lat']
customer_lon = df50['long']
store_lat = df50['merch_lat']
store_lon = df50['merch_long']
distances = haversine(customer_lat, customer_lon, store_lat, store_lon)
df50['distance_km'] = distances


In [23]:
df50 = df50[["amt",'trans_date_trans_time', 'distance_km', "is_fraud"]]

In [24]:
df50_tr,df50_test = sklearn.model_selection.train_test_split(df50, random_state=42)

## A. 데이터

In [26]:
tr = TabularDataset(df50_tr)
tst = TabularDataset(df50_test)

## B. predictor 생성

In [27]:
predictr = TabularPredictor("is_fraud")

No path specified. Models will be saved in: "AutogluonModels/ag-20231011_124455/"

## C.적합(fit)

In [28]:
predictr.fit(tr) 

Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231011_124455/"
AutoGluon Version:  0.8.2
Python Version:     3.8.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #26~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Jul 13 16:27:29 UTC 2
Disk Space Avail:   747.06 GB / 982.82 GB (76.0%)
Train Data Rows:    9009
Train Data Columns: 3
Label Column: is_fraud
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
    2 unique label values:  [1, 0]
    If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
    Available Memory:                    14638.47 MB
    Train Data (

In [29]:
predictr.leaderboard()

                  model  score_val  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2   0.927858       0.034292  9.131091                0.001279           0.502710            2       True         14
1              CatBoost   0.892342       0.002311  0.885833                0.002311           0.885833            1       True          7
2               XGBoost   0.892342       0.003765  0.143192                0.003765           0.143192            1       True         11
3              LightGBM   0.891232       0.002339  0.189846                0.002339           0.189846            1       True          4
4      RandomForestEntr   0.883463       0.029521  0.356271                0.029521           0.356271            1       True          6
5            LightGBMXT   0.881243       0.003287  0.271952                0.003287           0.271952            1       True          3
6         LightGBMLarge   0.876804